In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/allteamstable2", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/groupstagetable2", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/matchetable2", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/roundof_16", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/third_combo", True)


Out[121]: False

In [0]:
%sql
-- Create TeamTable
CREATE TABLE AllTeamsTable2 (
    TeamID INT,
    TeamName STRING,
    GroupName STRING
);

In [0]:
%sql
-- Insert data into TeamTable
INSERT INTO AllTeamsTable2 VALUES
(1, 'Germany', 'A'),
  (2, 'Scotland', 'A'),
  (3, 'Hungary', 'A'),
  (4, 'Switzerland', 'A'),
  (5, 'Spain', 'B'),
  (6, 'Croatia', 'B'),
  (7, 'Italy', 'B'),
  (8, 'Albania', 'B'),
  (9, 'Slovenia', 'C'),
  (10, 'Denmark', 'C'),
  (11, 'Serbia', 'C'),
  (12, 'England', 'C'),
  (13, 'Poland', 'D'),
  (14, 'Netherlands', 'D'),
  (15, 'Austria', 'D'),
  (16, 'France', 'D'),
  (17, 'Belgium', 'E'),
  (18, 'Slovakia', 'E'),
  (19, 'Romania', 'E'),
  (20, 'Ukraine', 'E'),
  (21, 'Türkiye', 'F'),
  (22, 'Greece', 'F'),
  (23, 'Portugal', 'F'),
  (24, 'Czech', 'F');


num_affected_rows,num_inserted_rows
24,24


In [0]:
%sql
select * from allteamstable2

TeamID,TeamName,GroupName
1,Germany,A
2,Scotland,A
3,Hungary,A
4,Switzerland,A
5,Spain,B
6,Croatia,B
7,Italy,B
8,Albania,B
9,Slovenia,C
10,Denmark,C


In [0]:
%sql
-- Create GroupStageTable
CREATE TABLE GroupStageTable2 (
    TeamName STRING,
    GroupName STRING,
    MatchesPlayed INT,
    Wins INT,
    Draws INT,
    Losses INT,
    GoalsFor INT,
    GoalsAgainst INT,
    GoalDifference INT,
    Points INT
);




In [0]:
%sql
CREATE TABLE matcheTable2 (
  MatchId INTEGER,
  Team1 VARCHAR(255),
  Team2 VARCHAR(255),
  Team1Score INTEGER,
  Team2Score INTEGER,
  Result VARCHAR(255),
  Stage VARCHAR(255)
);

In [0]:
%sql
INSERT INTO matcheTable2
SELECT
    ROW_NUMBER()OVER (ORDER BY t1.TeamId) AS MatchId,
    t1.TeamName AS Team1,
    t2.TeamName AS Team2,
    CAST(RAND() * 6 AS INT) AS Team1Score,
    CAST(RAND() * 6 AS INT) AS Team2Score,
    'draw' AS Result,
    'Group' AS Stage
FROM
    allteamstable2 t1
JOIN
    allteamstable2 t2
ON
    t1.TeamID > t2.TeamID
    AND t1.GroupName = t2.GroupName
    

num_affected_rows,num_inserted_rows
36,36


In [0]:
%sql
UPDATE matcheTable2
SET Result = 
    CASE
        WHEN Team1Score > Team2Score THEN Team1
        WHEN Team1Score < Team2Score THEN Team2
        ELSE 'draw'
    END;


select * from matcheTable2

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
1,Scotland,Germany,2,1,Scotland,Group
2,Hungary,Scotland,0,0,draw,Group
3,Hungary,Germany,3,0,Hungary,Group
4,Switzerland,Hungary,2,4,Hungary,Group
5,Switzerland,Scotland,3,0,Switzerland,Group
6,Switzerland,Germany,1,4,Germany,Group
7,Croatia,Spain,3,3,draw,Group
8,Italy,Croatia,1,0,Italy,Group
9,Italy,Spain,4,4,draw,Group
10,Albania,Italy,4,2,Albania,Group


In [0]:
%sql
INSERT INTO GroupStageTable2
SELECT
  Team as TeamName,
  GroupName as GroupName,
  COALESCE(MatchesPlayed, 0) AS MatchesPlayed,
  COALESCE(Wins, 0) AS Wins,
  COALESCE(Draws, 0) AS Draws,
  COALESCE(MatchesPlayed - COALESCE(Wins, 0) - COALESCE(Draws, 0), 0) AS Losses,
  COALESCE(GoalsFor, 0) AS GoalsFor,
  COALESCE(GoalsAgainst, 0) AS GoalsAgainst,
  COALESCE(GoalsFor - GoalsAgainst, 0) AS GoalDifference,
  COALESCE(Wins * 3 + Draws, 0) AS Points
FROM
  allteamstable2
LEFT JOIN (
  SELECT
  Team,
  sum(MatchesPlayed) as MatchesPlayed,
  sum(Wins) as Wins,
  sum(Draws) as Draws,
  Sum(GoalsFor) as GoalsFor,
  SUM(GoalsAgainst) AS GoalsAgainst
  From
  (SELECT
    Team1 as Team,
    COUNT(*) AS MatchesPlayed,
    SUM(CASE WHEN Result = Team1 THEN 1 ELSE 0 END) AS Wins,
    SUM(CASE WHEN Result = "draw" THEN 1 ELSE 0 END) AS Draws,
    SUM(Team1Score) AS GoalsFor,
    SUM(Team2Score) AS GoalsAgainst
  FROM
    matcheTable2
  WHERE
    Stage = 'Group'
  GROUP BY
    Team1

  UNION ALL

  SELECT
    Team2 as Team,

    COUNT(*) AS MatchesPlayed,
    SUM(CASE WHEN Result = Team2 THEN 1 ELSE 0 END) AS Wins,
    SUM(CASE WHEN Result = "draw" THEN 1 ELSE 0 END) AS Draws,
    SUM(Team1Score) AS GoalsFor,
    SUM(Team2Score) AS GoalsAgainst
  FROM
    matcheTable2
  WHERE
    Stage = 'Group'
  GROUP BY
    Team2
  )
  GROUP BY
    Team
) AS MatchStats
ON
  allteamstable2.TeamName = MatchStats.Team;


num_affected_rows,num_inserted_rows
24,24


In [0]:
%sql
select * from groupstagetable2

TeamName,GroupName,MatchesPlayed,Wins,Draws,Losses,GoalsFor,GoalsAgainst,GoalDifference,Points
Germany,A,3,1,0,2,6,5,1,3
Scotland,A,3,1,1,1,5,1,4,4
Hungary,A,3,2,1,0,5,4,1,7
Switzerland,A,3,1,0,2,6,8,-2,3
Spain,B,3,1,2,0,11,12,-1,5
Croatia,B,3,0,1,2,9,3,6,1
Italy,B,3,1,1,1,9,6,3,4
Albania,B,3,2,0,1,13,7,6,6
Slovenia,C,3,2,1,0,8,11,-3,7
Denmark,C,3,1,2,0,8,13,-5,5


In [0]:
%sql
CREATE TABLE roundOf_16 (
  TeamName VARCHAR(255),
  Group VARCHAR(255),
  Points INTEGER,
  RankInGroup INTEGER
);

In [0]:
%sql

WITH RankedTeams AS (
    SELECT
        G.TeamName,
        G.MatchesPlayed,
        G.Wins,
        G.Draws,
        G.Losses,
        G.GoalsFor,
        G.GoalsAgainst,
        G.GoalDifference,
        G.Points,
        Row_number() OVER (PARTITION BY T.GroupName ORDER BY G.Points DESC, G.GoalDifference DESC, G.GoalsFor DESC) AS RankInGroup
    FROM
        GroupStageTable2 G
        JOIN AllTeamsTable2 T ON G.TeamName = T.TeamName
)

Insert into roundOf_16 
    SELECT
        T.TeamName,
        T.GroupName,
        R.Points,
        R.RankInGroup
    FROM
        AllTeamsTable2 T
        JOIN RankedTeams R ON T.TeamName = R.TeamName
    WHERE
        R.RankInGroup <= 2;


num_affected_rows,num_inserted_rows
12,12


In [0]:
%sql
WITH RankedTeams AS (
    SELECT
        G.TeamName,
        G.MatchesPlayed,
        G.Wins,
        G.Draws,
        G.Losses,
        G.GoalsFor,
        G.GoalsAgainst,
        G.GoalDifference,
        G.Points,
        T.GroupName,
        Row_number() OVER (PARTITION BY T.GroupName ORDER BY G.Points DESC, G.GoalDifference DESC, G.GoalsFor DESC) AS RankInGroup
    FROM
        GroupStageTable2 G
        JOIN AllTeamsTable2 T ON G.TeamName = T.TeamName
),
RankedThirdPlacedTeams AS (
    SELECT
        TeamName,
        GroupName,
        Points,
        Row_number() OVER (ORDER BY Points DESC, GoalDifference DESC, GoalsFor DESC) AS RankThirdPlaced
    FROM
        RankedTeams
    WHERE
        RankInGroup = 3
)

INSERT INTO roundOf_16 
SELECT
    TeamName,
    GroupName,
    Points,
    3
FROM
    RankedThirdPlacedTeams
WHERE
    RankThirdPlaced <= 4;


num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
select * from roundof_16

TeamName,Group,Points,RankInGroup
Hungary,A,7,1
Scotland,A,4,2
Albania,B,6,1
Spain,B,5,2
Slovenia,C,7,1
Denmark,C,5,2
France,D,7,1
Poland,D,5,2
Romania,E,6,1
Ukraine,E,5,2


# Third Combination Table
The matches for Round of 16 is decided according to the combination retrieved from the following 15 possible combinations.

In [0]:
%sql
CREATE TABLE third_combo (
  ID INTEGER,
  SELECTEDGROUPS VARCHAR(4),
  Team1B CHAR(1),
  Team1C CHAR(1),
  Team1E CHAR(1),
  Team1F CHAR(1)
);

INSERT INTO THIRD_COMBO (ID, SELECTEDGROUPS, Team1B, Team1C, Team1E, Team1F) VALUES
(1, 'ABCD', 'A', 'D', 'B', 'C'),
(2, 'ABCE', 'A', 'E', 'B', 'C'),
(3, 'ABCF', 'A', 'F', 'B', 'C'),
(4, 'ABDE', 'D', 'E', 'A', 'B'),
(5, 'ABDF', 'D', 'F', 'A', 'B'),
(6, 'ABEF', 'E', 'F', 'B', 'A'),
(7, 'ACDE', 'E', 'D', 'C', 'A'),
(8, 'ACDF', 'F', 'D', 'C', 'A'),
(9, 'ACEF', 'E', 'F', 'C', 'A'),
(10, 'ADEF', 'E', 'F', 'D', 'A'),
(11, 'BCDE', 'E', 'D', 'B', 'C'),
(12, 'BCDF', 'F', 'D', 'C', 'B'),
(13, 'BCEF', 'F', 'E', 'C', 'B'),
(14, 'BDEF', 'F', 'E', 'D', 'B'),
(15, 'CDEF', 'F', 'E', 'D', 'C');


select * from THIRD_COMBO


ID,SELECTEDGROUPS,Team1B,Team1C,Team1E,Team1F
1,ABCD,A,D,B,C
2,ABCE,A,E,B,C
3,ABCF,A,F,B,C
4,ABDE,D,E,A,B
5,ABDF,D,F,A,B
6,ABEF,E,F,B,A
7,ACDE,E,D,C,A
8,ACDF,F,D,C,A
9,ACEF,E,F,C,A
10,ADEF,E,F,D,A


# Round of 16 matches

In [0]:
%sql

insert into matchetable2(MatchId,
  Team1,
  Team2,
  Team1Score,
  Team2Score,
  Result,
  Stage)
With selectedgroup as 
(
    select
    CONCAT_WS('', SORT_ARRAY(COLLECT_SET(SUBSTRING(Group, 1, 1)))) AS Result
    from roundof_16
    where RankInGroup = 3

),

Selectedgrouptable as
(
  select * 
  from third_combo
  where third_combo.selectedgroups = (select result from selectedgroup)
)

SELECT
    row_number() over(order by 1) + 36 as MatchId,
    r1.TeamName as Team1,
    r2.TeamName as Team2,
    CAST(RAND() * 6 AS INT) AS Team1Score,
    CAST(RAND() * 6 AS INT) AS Team2Score,
    'draw' AS Result,
    'Round of 16' AS Stage
FROM
    roundof_16 r1
JOIN
    roundof_16 r2
ON 
    (r1.group = 'B' AND r1.rankingroup = 1 AND r2.group = (SELECT Team1B FROM selectedgrouptable LIMIT 1) AND r2.rankingroup = 3)
UNION ALL

SELECT
    row_number() over(order by 1) + 37 as MatchId,
    r1.TeamName as Team1,
    r2.TeamName as Team2,
    CAST(RAND() * 6 AS INT) AS Team1Score,
    CAST(RAND() * 6 AS INT) AS Team2Score,
    'draw' AS Result,
    'Round of 16' AS Stage
FROM
    roundof_16 r1
JOIN
    roundof_16 r2
ON 
    (r1.group = 'A' AND r1.rankingroup = 1 AND r2.group = 'C' AND r2.rankingroup = 2) 
UNION ALL

SELECT
    row_number() over(order by 1) + 38 as MatchId,
    r1.TeamName as Team1,
    r2.TeamName as Team2,
    CAST(RAND() * 6 AS INT) AS Team1Score,
    CAST(RAND() * 6 AS INT) AS Team2Score,
    'draw' AS Result,
    'Round of 16' AS Stage
FROM
    roundof_16 r1
JOIN
    roundof_16 r2
ON 
    (r1.group = 'F' AND r1.rankingroup = 1 AND r2.group = (SELECT Team1F FROM selectedgrouptable LIMIT 1) AND r2.rankingroup = 3) 

UNION ALL

SELECT
    row_number() over(order by 1) + 39 as MatchId,
    r1.TeamName as Team1,
    r2.TeamName as Team2,
    CAST(RAND() * 6 AS INT) AS Team1Score,
    CAST(RAND() * 6 AS INT) AS Team2Score,
    'draw' AS Result,
    'Round of 16' AS Stage
FROM
    roundof_16 r1
JOIN
    roundof_16 r2
ON 
    (r1.group = 'D' AND r1.rankingroup = 2 AND r2.group = 'E' AND r2.rankingroup = 2)     
UNION ALL

SELECT
    row_number() over(order by 1) + 40 as MatchId,
    r1.TeamName as Team1,
    r2.TeamName as Team2,
    CAST(RAND() * 6 AS INT) AS Team1Score,
    CAST(RAND() * 6 AS INT) AS Team2Score,
    'draw' AS Result,
    'Round of 16' AS Stage
FROM
    roundof_16 r1
JOIN
    roundof_16 r2
ON 
    (r1.group = 'E' AND r1.rankingroup = 1 AND r2.group = (SELECT Team1E FROM selectedgrouptable LIMIT 1) AND r2.rankingroup = 3)
UNION ALL

SELECT
    row_number() over(order by 1) + 41 as MatchId,
    r1.TeamName as Team1,
    r2.TeamName as Team2,
    CAST(RAND() * 6 AS INT) AS Team1Score,
    CAST(RAND() * 6 AS INT) AS Team2Score,
    'draw' AS Result,
    'Round of 16' AS Stage
FROM
    roundof_16 r1
JOIN
    roundof_16 r2
ON 
    (r1.group = 'D' AND r1.rankingroup = 1 AND r2.group = 'F' AND r2.rankingroup = 2)
UNION ALL

SELECT
    row_number() over(order by 1) + 42 as MatchId,
    r1.TeamName as Team1,
    r2.TeamName as Team2,
    CAST(RAND() * 6 AS INT) AS Team1Score,
    CAST(RAND() * 6 AS INT) AS Team2Score,
    'draw' AS Result,
    'Round of 16' AS Stage
FROM
    roundof_16 r1
JOIN
    roundof_16 r2
ON 
    (r1.group = 'C' AND r1.rankingroup = 1 AND r2.group = (SELECT Team1C FROM selectedgrouptable LIMIT 1) AND r2.rankingroup = 3)
UNION ALL

SELECT
    row_number() over(order by 1) + 43 as MatchId,
    r1.TeamName as Team1,
    r2.TeamName as Team2,
    CAST(RAND() * 6 AS INT) AS Team1Score,
    CAST(RAND() * 6 AS INT) AS Team2Score,
    'draw' AS Result,
    'Round of 16' AS Stage
FROM
    roundof_16 r1
JOIN
    roundof_16 r2
ON 
    (r1.group = 'A' AND r1.rankingroup = 2 AND r2.group = 'B' AND r2.rankingroup = 2);

UPDATE matcheTable2
SET Result = 
    CASE
        WHEN Team1Score > Team2Score THEN Team1
        WHEN Team1Score < Team2Score THEN Team2
        ELSE 
            CASE
                WHEN (Team1Score) % 2 = 1 THEN Team1
                ELSE Team2
            END  
    END
WHERE Stage = 'Round of 16';



num_affected_rows
8


In [0]:
%sql
select * from matchetable2
ORDER BY MATCHID

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
1,Scotland,Germany,2,1,Scotland,Group
2,Hungary,Scotland,0,0,draw,Group
3,Hungary,Germany,3,0,Hungary,Group
4,Switzerland,Hungary,2,4,Hungary,Group
5,Switzerland,Scotland,3,0,Switzerland,Group
6,Switzerland,Germany,1,4,Germany,Group
7,Croatia,Spain,3,3,draw,Group
8,Italy,Croatia,1,0,Italy,Group
9,Italy,Spain,4,4,draw,Group
10,Albania,Italy,4,2,Albania,Group


# Quarter Final Matches


In [0]:
%sql
insert into matchetable2(MatchId,
  Team1,
  Team2,
  Team1Score,
  Team2Score,
  Result,
  Stage)
SELECT 
    45 AS MatchId, 
    (SELECT Result FROM matchetable2 WHERE MatchId = 37) AS team1, 
    (SELECT Result FROM matchetable2 WHERE MatchId = 38) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Quarter Final' as stage
UNION ALL
SELECT 
    46 AS MatchId, 
    (SELECT Result FROM matchetable2 WHERE MatchId = 39) AS team1, 
    (SELECT Result FROM matchetable2 WHERE MatchId = 40) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Quarter Final' as stage
UNION ALL
SELECT 
    47 AS MatchId, 
    (SELECT Result FROM matchetable2 WHERE MatchId = 41) AS team1, 
    (SELECT Result FROM matchetable2 WHERE MatchId = 42) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Quarter Final' as stage
UNION ALL
SELECT 
    48 AS MatchId, 
    (SELECT Result FROM matchetable2 WHERE MatchId = 43) AS team1, 
    (SELECT Result FROM matchetable2 WHERE MatchId = 44) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Quarter Final' as stage;

UPDATE matchetable2
SET Result = 
    CASE 
        WHEN team1score > team2score THEN team1
        WHEN team1score < team2score THEN team2
        ELSE 
            CASE 
                WHEN (team1score) % 2 = 1 THEN team1
                ELSE team2
            END
    END
where stage = 'Quarter Final';

select * from matchetable2
order by MatchId

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
1,Scotland,Germany,2,1,Scotland,Group
2,Hungary,Scotland,0,0,draw,Group
3,Hungary,Germany,3,0,Hungary,Group
4,Switzerland,Hungary,2,4,Hungary,Group
5,Switzerland,Scotland,3,0,Switzerland,Group
6,Switzerland,Germany,1,4,Germany,Group
7,Croatia,Spain,3,3,draw,Group
8,Italy,Croatia,1,0,Italy,Group
9,Italy,Spain,4,4,draw,Group
10,Albania,Italy,4,2,Albania,Group


# Semi Final Matches

In [0]:
%sql
insert into matchetable2(MatchId,
  Team1,
  Team2,
  Team1Score,
  Team2Score,
  Result,
  Stage)
SELECT 
    49 AS MatchId, 
    (SELECT Result FROM matchetable2 WHERE MatchId = 45) AS team1, 
    (SELECT Result FROM matchetable2 WHERE MatchId = 46) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Semi Final' as stage
UNION ALL
SELECT 
    50 AS MatchId, 
    (SELECT Result FROM matchetable2 WHERE MatchId = 47) AS team1, 
    (SELECT Result FROM matchetable2 WHERE MatchId = 48) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Semi Final' as stage;

UPDATE matchetable2
SET Result = 
    CASE 
        WHEN team1score > team2score THEN team1
        WHEN team1score < team2score THEN team2
        ELSE 
            CASE 
                WHEN (team1score) % 2 = 1 THEN team1
                ELSE team2
            END
    END
where stage = 'Semi Final';

select * from matchetable2
order by MatchId

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
1,Scotland,Germany,2,1,Scotland,Group
2,Hungary,Scotland,0,0,draw,Group
3,Hungary,Germany,3,0,Hungary,Group
4,Switzerland,Hungary,2,4,Hungary,Group
5,Switzerland,Scotland,3,0,Switzerland,Group
6,Switzerland,Germany,1,4,Germany,Group
7,Croatia,Spain,3,3,draw,Group
8,Italy,Croatia,1,0,Italy,Group
9,Italy,Spain,4,4,draw,Group
10,Albania,Italy,4,2,Albania,Group


# Third Place Match

In [0]:
%sql
INSERT INTO matchetable2 (MatchId, team1, team2, team1score, team2score, Result, stage)
SELECT 
    51 AS MatchId, 
    (SELECT Result FROM matchetable2 WHERE stage='Quarter Final' and Result not in (select Result FROM matchetable2 WHERE stage='Semi Final') LIMIT 1 OFFSET 0) AS team1, 
    (SELECT Result FROM matchetable2 WHERE stage='Quarter Final' and Result not in (select Result FROM matchetable2 WHERE stage='Semi Final') LIMIT 1 OFFSET 1) AS team2,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team1Score,
    (SELECT FLOOR(RAND() * 6) + 1) AS Team2Score,
    'draw' as Result,
    'Third Place' as stage;
UPDATE matchetable2
SET Result = 
    CASE 
        WHEN team1score > team2score THEN team1
        WHEN team1score < team2score THEN team2
        ELSE 
            CASE 
                WHEN (team1score) % 2 = 1 THEN team1
                ELSE team2
            END
    END
where stage = 'Third Place';

select * from matchetable2
order by MatchId

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
1,Scotland,Germany,2,1,Scotland,Group
2,Hungary,Scotland,0,0,draw,Group
3,Hungary,Germany,3,0,Hungary,Group
4,Switzerland,Hungary,2,4,Hungary,Group
5,Switzerland,Scotland,3,0,Switzerland,Group
6,Switzerland,Germany,1,4,Germany,Group
7,Croatia,Spain,3,3,draw,Group
8,Italy,Croatia,1,0,Italy,Group
9,Italy,Spain,4,4,draw,Group
10,Albania,Italy,4,2,Albania,Group


# Final Match


In [0]:
%sql
insert into matchetable2(MatchId,
  Team1,
  Team2,
  Team1Score,
  Team2Score,
  Result,
  Stage)

  SELECT 
    52 AS MatchId, 
    f1.Result AS team1, 
    f2.Result AS team2,
    FLOOR(RAND() * 6) + 1 AS Team1Score,
    FLOOR(RAND() * 6) + 1 AS Team2Score,
    'draw' as Result,
    'Final' as stage
  from
    matchetable2 f1
  join
    matchetable2 f2
  on 
    f1.MatchID = 49 and f2.MatchID = 50;
  
UPDATE matchetable2
SET Result = 
    CASE 
        WHEN team1score > team2score THEN team1
        WHEN team1score < team2score THEN team2
        ELSE 
            CASE 
                WHEN (team1score) % 2 = 1 THEN team1
                ELSE team2
            END
    END
where stage = 'Final';

select * from matchetable2
order by MatchId

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
1,Scotland,Germany,2,1,Scotland,Group
2,Hungary,Scotland,0,0,draw,Group
3,Hungary,Germany,3,0,Hungary,Group
4,Switzerland,Hungary,2,4,Hungary,Group
5,Switzerland,Scotland,3,0,Switzerland,Group
6,Switzerland,Germany,1,4,Germany,Group
7,Croatia,Spain,3,3,draw,Group
8,Italy,Croatia,1,0,Italy,Group
9,Italy,Spain,4,4,draw,Group
10,Albania,Italy,4,2,Albania,Group


In [0]:
%sql
select * from matchetable2
order by matchid

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
1,Scotland,Germany,2,1,Scotland,Group
2,Hungary,Scotland,0,0,draw,Group
3,Hungary,Germany,3,0,Hungary,Group
4,Switzerland,Hungary,2,4,Hungary,Group
5,Switzerland,Scotland,3,0,Switzerland,Group
6,Switzerland,Germany,1,4,Germany,Group
7,Croatia,Spain,3,3,draw,Group
8,Italy,Croatia,1,0,Italy,Group
9,Italy,Spain,4,4,draw,Group
10,Albania,Italy,4,2,Albania,Group


In [0]:
%sql
select * from matchetable2 where stage = "Round of 16"
order by MatchId

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
37,Albania,Portugal,1,0,Albania,Round of 16
38,Hungary,Denmark,0,3,Denmark,Round of 16
39,Czech,Italy,1,1,Czech,Round of 16
40,Poland,Ukraine,0,4,Ukraine,Round of 16
41,Romania,England,0,1,England,Round of 16
42,France,Türkiye,1,1,France,Round of 16
43,Slovenia,Belgium,1,3,Belgium,Round of 16
44,Scotland,Spain,5,4,Scotland,Round of 16


In [0]:
%sql
select * from matchetable2 where stage = "Quater Final" 
order by MatchId

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage


In [0]:
%sql
select * from matchetable2 where stage = "Semi Final"
order by MatchId

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
49,Albania,Ukraine,2,2,Ukraine,Semi Final
50,France,Belgium,1,6,Belgium,Semi Final


In [0]:
%sql
select * from matchetable2 where stage = "Third Place"
order by MatchId

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
51,Albania,France,2,5,France,Third Place


In [0]:
%sql
select * from matchetable2 where stage = "Final"
order by MatchId

MatchId,Team1,Team2,Team1Score,Team2Score,Result,Stage
52,Ukraine,Belgium,3,1,Ukraine,Final


# Euro Cup 2024 Visualization

In [0]:
#import libraries
import pandas as pd
from pyspark.sql import SparkSession

In [0]:
#Spark Session
spark = SparkSession.builder \
    .appName("euro_cup_visulization") \
    .getOrCreate()

In [0]:
groupstage = spark.sql("SELECT * from groupstagetable2")
round_of_16 = spark.sql("SELECT Team1,Team2,Team1score,Team2score FROM Matchetable2 WHERE Stage = 'Round of 16' ORDER BY MatchId")
quater_final = spark.sql("SELECT Team1,Team2,Team1score,Team2score FROM Matchetable2 WHERE Stage = 'Quarter Final' ORDER BY MatchId")
semi_final = spark.sql("SELECT Team1,Team2,Team1score,Team2score FROM Matchetable2 WHERE Stage = 'Semi Final' ORDER BY MatchId")
final = spark.sql("SELECT Team1,Team2,Team1score,Team2score FROM Matchetable2 WHERE Stage = 'Final' ORDER BY MatchId")
third_place = spark.sql("SELECT MatchId  FROM Matchetable2 WHERE Stage = 'Third Place' ORDER BY MatchId")


In [0]:
groupstage_df = groupstage.toPandas()
round_of_16_df = round_of_16.toPandas()
quater_final_df = quater_final.toPandas()
semi_final_df = semi_final.toPandas()
final_df = final.toPandas()
third_place_df = third_place.toPandas()

In [0]:
table_visualization = pd.DataFrame({
    'Teams': groupstage_df['TeamName'],
    'Group': groupstage_df['GroupName'],
    'Pld': groupstage_df['MatchesPlayed'],
    'W': groupstage_df['Wins'],
    'D': groupstage_df['Draws'],
    'L': groupstage_df['Losses'],
    'GF': groupstage_df['GoalsFor'],
    'GA': groupstage_df['GoalsAgainst'],
    'GD': groupstage_df['GoalDifference'],
    'Pts': groupstage_df['Points']
})

In [0]:
# Create blank row as a dictionary to add one blank row for each group
blank_row = {
    'Teams': '', 
    'Group': '', 
    'Pld': '', 
    'W': '',
    'D': '',
    'L': '',
    'GF': '',
    'GA': '',
    'GD': '',
    'Pts': ''
}

In [0]:
groups = []
for i in range(0, len(table_visualization), 4):
    group = table_visualization.iloc[i:i+4]  
    groups.append(group) 
    if i < len(table_visualization) - 4:
        groups.append(pd.DataFrame([blank_row])) 

visualization_with_space = pd.concat(groups).reset_index(drop=True)
half_table_index = len(visualization_with_space) // 2
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
display(visualization_with_space)
print(half_table_index)

Teams,Group,Pld,W,D,L,GF,GA,GD,Pts
Germany,A,3,1,0,2,6,5,1,3
Scotland,A,3,1,1,1,5,1,4,4
Hungary,A,3,2,1,0,5,4,1,7
Switzerland,A,3,1,0,2,6,8,-2,3
,,,,,,,,,
Spain,B,3,1,2,0,11,12,-1,5
Croatia,B,3,0,1,2,9,3,6,1
Italy,B,3,1,1,1,9,6,3,4
Albania,B,3,2,0,1,13,7,6,6
,,,,,,,,,


14


In [0]:
#Table information for the Group A,B,C
left_table = pd.DataFrame(columns=['Teams','Group','Pld','W','D','L','GF','GA','GD','Pts'])
left_table = pd.concat([left_table, pd.DataFrame([blank_row])], ignore_index=True)
left_table = pd.concat([left_table, visualization_with_space.iloc[:half_table_index]]).reset_index(drop=True)

#Table information for the group C,D,E
right_table = pd.DataFrame(columns=['Teams','Group','Pld','W','D','L','GF','GA','GD','Pts'])
right_table = pd.concat([right_table, visualization_with_space.iloc[half_table_index +1:]]).reset_index(drop=True)
right_table = pd.concat([pd.DataFrame([blank_row]), right_table], ignore_index=True)

display(left_table)
display(right_table)

Teams,Group,Pld,W,D,L,GF,GA,GD,Pts
,,,,,,,,,
Germany,A,3,1,0,2,6,5,1,3
Scotland,A,3,1,1,1,5,1,4,4
Hungary,A,3,2,1,0,5,4,1,7
Switzerland,A,3,1,0,2,6,8,-2,3
,,,,,,,,,
Spain,B,3,1,2,0,11,12,-1,5
Croatia,B,3,0,1,2,9,3,6,1
Italy,B,3,1,1,1,9,6,3,4
Albania,B,3,2,0,1,13,7,6,6


Teams,Group,Pld,W,D,L,GF,GA,GD,Pts
,,,,,,,,,
Poland,D,3,1,2,0,2,6,-4,5
Netherlands,D,3,0,1,2,9,1,8,1
Austria,D,3,1,0,2,10,9,1,3
France,D,3,2,1,0,9,4,5,7
,,,,,,,,,
Belgium,E,3,1,1,1,4,7,-3,4
Slovakia,E,3,0,1,2,7,7,0,1
Romania,E,3,2,0,1,11,4,7,6
Ukraine,E,3,1,2,0,10,8,2,5


In [0]:
visualize_r16 = pd.DataFrame(columns=['Round_of_16', 'rest'])

for _ in range(2):
    visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': ['']})], ignore_index=True)

for index, row in round_of_16_df.iterrows():
    visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': [f"{row['Team1']} ({row['Team1score']})"]})], ignore_index=True)
    visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': [f"{row['Team2']} ({row['Team2score']})"]})], ignore_index=True)

    for _ in range(2):
        visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': ['']})], ignore_index=True)

visualize_r16['rest'] = ''

splited_r16_index = len(visualize_r16) // 2
print(splited_r16_index)
display(visualize_r16)

17


Round_of_16,rest
,
,
Albania (1),
Portugal (0),
,
,
Hungary (0),
Denmark (3),
,
,


In [0]:
left_side_r16 = visualize_r16.iloc[:splited_r16_index + 1].reset_index(drop = True)
right_side_r16 = visualize_r16.iloc[splited_r16_index - 1:].reset_index(drop = True)
display(left_side_r16)
display(right_side_r16)

Round_of_16,rest
,
,
Albania (1),
Portugal (0),
,
,
Hungary (0),
Denmark (3),
,
,


Round_of_16,rest
,
,
Romania (0),
England (1),
,
,
France (1),
Türkiye (1),
,
,


In [0]:
visualize_qf = pd.DataFrame(columns=['Quater_Final'])

for _ in range(4):
    visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'Quater_Final': ['']})], ignore_index=True)

for index, row in quater_final_df.iterrows():
    visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'Quater_Final': [f"{row['Team1']} ({row['Team1score']})"]})], ignore_index=True)
    visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'Quater_Final': [f"{row['Team2']} ({row['Team2score']})"]})], ignore_index=True)

    for _ in range(6):
        visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'Quater_Final': ['']})], ignore_index=True)
visualize_qf['rest'] = ''
splited_qf_index = len(visualize_qf) // 2
display(visualize_qf)

Quater_Final,rest
,
,
,
,
Albania (6),
Denmark (5),
,
,
,
,


In [0]:
left_side_qf = pd.DataFrame(columns=['Quater_Final'])
left_side_qf = pd.concat([left_side_qf,visualize_qf.iloc[:splited_qf_index]]).reset_index(drop=True)
right_side_qf = pd.DataFrame(columns=['Quater_Final'])
right_side_qf = pd.concat([right_side_qf,visualize_qf.iloc[splited_qf_index-2:]]).reset_index(drop=True)


display(left_side_qf)
display(right_side_qf)

Quater_Final,rest
,
,
,
,
Albania (6),
Denmark (5),
,
,
,
,


Quater_Final,rest
,
,
,
,
England (3),
France (4),
,
,
,
,


In [0]:
visualize_sf = pd.DataFrame(columns=['Semi_Final'])

for _ in range(8):
    visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'Semi_Final': ['']})], ignore_index=True)

for index, row in semi_final_df.iterrows():
    visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'Semi_Final': [f"{row['Team1']} ({row['Team1score']})"]})], ignore_index=True)
    visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'Semi_Final': [f"{row['Team2']} ({row['Team2score']})"]})], ignore_index=True)
    for _ in range(6):
        visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'Semi_Final': ['']})], ignore_index=True)
visualize_sf['rest'] = ''
splited_sf_index = len(visualize_sf) // 2
print(splited_sf_index)
display(visualize_sf)


12


Semi_Final,rest
,
,
,
,
,
,
,
,
Albania (2),
Ukraine (2),


In [0]:
left_side_sf = pd.DataFrame(columns=['Semi_Final'])
left_side_sf = pd.concat([left_side_sf, visualize_sf.iloc[:splited_sf_index]]).reset_index(drop=True)

for _ in range(5):
    left_side_sf = pd.concat([left_side_sf, pd.DataFrame({'Semi_Final': [''], 'rest': ['']})], ignore_index=True)

right_side_sf = pd.DataFrame({'Semi_Final': [''] * 7, 'rest': [''] * 7})
right_side_sf = pd.concat([right_side_sf, visualize_sf.iloc[splited_sf_index+3:]]).reset_index(drop=True)
display(left_side_sf)
display(right_side_sf)
     

Semi_Final,rest
,
,
,
,
,
,
,
,
Albania (2),
Ukraine (2),


Semi_Final,rest
,
,
,
,
,
,
,
,
France (1),
Belgium (6),


In [0]:

visualize_final = pd.DataFrame(columns=['Final'])

for _ in range(8):
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': ['']})], ignore_index=True)

for index, row in final_df.iterrows():
    winner = ''
    
    if row['Team1score'] > row['Team2score']:
        winner = row['Team1']
    else:
        winner = row['Team2']

    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': [f"{row['Team1']} ({row['Team1score']})"]})], ignore_index=True)
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': [f"{row['Team2']} ({row['Team2score']})"]})], ignore_index=True)
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': [f"Winner: {winner}"]})], ignore_index=True)

visualize_final['rest']= ''
display(visualize_final)


Final,rest
,
,
,
,
,
,
,
,
Ukraine (3),
Belgium (1),


In [0]:
left_side_qf.columns = [f"{col}q2" for col in left_side_qf.columns]

merged_df = pd.concat([left_side_r16, left_side_qf], axis=1)

left_side_sf.columns = [f"{col}s1" for col in left_side_sf.columns]
merged_df = pd.concat([merged_df, left_side_sf], axis=1)

visualize_final.columns = [f"{col}f" for col in visualize_final.columns]
merged_df = pd.concat([merged_df, visualize_final], axis=1)

right_side_sf.columns = [f"{col}s2" for col in right_side_sf.columns]

merged_df = pd.concat([merged_df, right_side_sf], axis=1)



right_side_qf.columns = [f"{col}q1" for col in right_side_qf.columns]
# Concatenate the DataFrames
merged_df = pd.concat([merged_df, right_side_qf], axis=1)

right_side_r16.columns = [f"{col}r2" for col in right_side_r16.columns]

# Concatenate the DataFrames
merged_df = pd.concat([merged_df,right_side_r16], axis=1)

left_table.columns = [f"{col}g1" for col in left_table.columns]
right_table.columns = [f"{col}g2" for col in right_table.columns]

# Concatenate the DataFrames
merged_df = pd.concat([left_table,merged_df, right_table], axis=1)

merged_df.fillna('', inplace=True)

display(merged_df)

Teamsg1,Groupg1,Pldg1,Wg1,Dg1,Lg1,GFg1,GAg1,GDg1,Ptsg1,Round_of_16,rest,Quater_Finalq2,restq2,Semi_Finals1,rests1,Finalf,restf,Semi_Finals2,rests2,Quater_Finalq1,restq1,Round_of_16r2,restr2,Teamsg2,Groupg2,Pldg2,Wg2,Dg2,Lg2,GFg2,GAg2,GDg2,Ptsg2
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Germany,A,3,1,0,2,6,5,1,3,,,,,,,,,,,,,,,Poland,D,3,1,2,0,2,6,-4,5
Scotland,A,3,1,1,1,5,1,4,4,Albania (1),,,,,,,,,,,,Romania (0),,Netherlands,D,3,0,1,2,9,1,8,1
Hungary,A,3,2,1,0,5,4,1,7,Portugal (0),,,,,,,,,,,,England (1),,Austria,D,3,1,0,2,10,9,1,3
Switzerland,A,3,1,0,2,6,8,-2,3,,,Albania (6),,,,,,,,England (3),,,,France,D,3,2,1,0,9,4,5,7
,,,,,,,,,,,,Denmark (5),,,,,,,,France (4),,,,,,,,,,,,,
Spain,B,3,1,2,0,11,12,-1,5,Hungary (0),,,,,,,,,,,,France (1),,Belgium,E,3,1,1,1,4,7,-3,4
Croatia,B,3,0,1,2,9,3,6,1,Denmark (3),,,,,,,,,,,,Türkiye (1),,Slovakia,E,3,0,1,2,7,7,0,1
Italy,B,3,1,1,1,9,6,3,4,,,,,Albania (2),,Ukraine (3),,France (1),,,,,,Romania,E,3,2,0,1,11,4,7,6
Albania,B,3,2,0,1,13,7,6,6,,,,,Ukraine (2),,Belgium (1),,Belgium (6),,,,,,Ukraine,E,3,1,2,0,10,8,2,5


In [0]:
# Concatenate the DataFrames
merged_df = pd.concat([left_table, left_side_r16, left_side_qf, left_side_sf, visualize_final, right_side_sf, right_side_qf, right_side_r16, right_table], axis=1)
merged_df.fillna('', inplace=True)
# Display the DataFrame
display(merged_df)


Teamsg1,Groupg1,Pldg1,Wg1,Dg1,Lg1,GFg1,GAg1,GDg1,Ptsg1,Round_of_16,rest,Quater_Finalq2,restq2,Semi_Finals1,rests1,Finalf,restf,Semi_Finals2,rests2,Quater_Finalq1,restq1,Round_of_16r2,restr2,Teamsg2,Groupg2,Pldg2,Wg2,Dg2,Lg2,GFg2,GAg2,GDg2,Ptsg2
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Germany,A,3,1,0,2,6,5,1,3,,,,,,,,,,,,,,,Poland,D,3,1,2,0,2,6,-4,5
Scotland,A,3,1,1,1,5,1,4,4,Albania (1),,,,,,,,,,,,Romania (0),,Netherlands,D,3,0,1,2,9,1,8,1
Hungary,A,3,2,1,0,5,4,1,7,Portugal (0),,,,,,,,,,,,England (1),,Austria,D,3,1,0,2,10,9,1,3
Switzerland,A,3,1,0,2,6,8,-2,3,,,Albania (6),,,,,,,,England (3),,,,France,D,3,2,1,0,9,4,5,7
,,,,,,,,,,,,Denmark (5),,,,,,,,France (4),,,,,,,,,,,,,
Spain,B,3,1,2,0,11,12,-1,5,Hungary (0),,,,,,,,,,,,France (1),,Belgium,E,3,1,1,1,4,7,-3,4
Croatia,B,3,0,1,2,9,3,6,1,Denmark (3),,,,,,,,,,,,Türkiye (1),,Slovakia,E,3,0,1,2,7,7,0,1
Italy,B,3,1,1,1,9,6,3,4,,,,,Albania (2),,Ukraine (3),,France (1),,,,,,Romania,E,3,2,0,1,11,4,7,6
Albania,B,3,2,0,1,13,7,6,6,,,,,Ukraine (2),,Belgium (1),,Belgium (6),,,,,,Ukraine,E,3,1,2,0,10,8,2,5
